In [1]:
import numpy as np
import nltk, os
from word import Word

In [2]:
class i2b2Parser():
    filepath =  None
    tokens = None

    def __init__(self, filepath):
        self.filepath = filepath

    def parse_file(self):
        with open(self.filepath, 'r') as f:
            data = f.read().splitlines()
        tokens = []
        for linenum, line in enumerate(data):
            for tokennum, token in enumerate(line.split()):
                token = {'text': token, 'token': tokennum, 'line': linenum + 1}
                tokens.append(token)
        self.tokens = tokens
    
    def tag_gold_tokens

# Testing

In [3]:
colab_path = os.environ['colab']
data_path = colab_path + '/data/i2b2/2009 Medication Challenge/'
sample_file = data_path + '/training_sets_released/2/11995'

In [4]:
parser = i2b2Parser(sample_file)
parser.parse_file()
tokens = parser.tokens

In [6]:
for token in tokens:
   if token['line'] == 38:
        print(token)

{'text': 'p.o.', 'token': 0, 'line': 38}
{'text': 'q', 'token': 1, 'line': 38}
{'text': '12', 'token': 2, 'line': 38}
{'text': ',', 'token': 3, 'line': 38}
{'text': 'Ecotrin', 'token': 4, 'line': 38}
{'text': '225', 'token': 5, 'line': 38}
{'text': 'mg', 'token': 6, 'line': 38}
{'text': 'q', 'token': 7, 'line': 38}
{'text': 'd', 'token': 8, 'line': 38}
{'text': ',', 'token': 9, 'line': 38}
{'text': 'Vancomycin', 'token': 10, 'line': 38}
{'text': '1250', 'token': 11, 'line': 38}
{'text': 'mg', 'token': 12, 'line': 38}
{'text': 'q', 'token': 13, 'line': 38}
{'text': '24.', 'token': 14, 'line': 38}
{'text': 'She', 'token': 15, 'line': 38}
